In [1]:
from selenium import webdriver  
from bs4 import BeautifulSoup
import requests
tripadvisorca = "https://www.tripadvisor.ca"
region_link = "https://www.tripadvisor.ca/Hotels-g154913-Calgary_Alberta-Hotels.html"
region_ID = "-g154913-"

In [2]:
def region_urls(first_url, region_ID="-g154913-"):
    result = requests.get(first_url)
    content = result.content
    soup = BeautifulSoup(content, "html.parser")
    num_pages = int(soup.find_all("a", attrs={"class":"pageNum last taLnk "})[0].get_text(strip=True))
    list_urls = []
    for i in range(num_pages):
        bn = region_ID + "oa" + str(i * 30) + "-"
        url = first_url.replace(region_ID, bn)
        list_urls.append(url)
    return list_urls
region_links = region_urls(region_link)
region_links

['https://www.tripadvisor.ca/Hotels-g154913-oa0-Calgary_Alberta-Hotels.html',
 'https://www.tripadvisor.ca/Hotels-g154913-oa30-Calgary_Alberta-Hotels.html',
 'https://www.tripadvisor.ca/Hotels-g154913-oa60-Calgary_Alberta-Hotels.html',
 'https://www.tripadvisor.ca/Hotels-g154913-oa90-Calgary_Alberta-Hotels.html',
 'https://www.tripadvisor.ca/Hotels-g154913-oa120-Calgary_Alberta-Hotels.html',
 'https://www.tripadvisor.ca/Hotels-g154913-oa150-Calgary_Alberta-Hotels.html',
 'https://www.tripadvisor.ca/Hotels-g154913-oa180-Calgary_Alberta-Hotels.html']

In [3]:
def hotel_urls(list_urls):
    list_hotel = []
    for page in list_urls:
        result = requests.get(page)
        content = result.content
        soup = BeautifulSoup(content, "html.parser")
        
        urls = soup.find_all("a", attrs={"class": "review_count"})
        names = soup.find_all("a", attrs={"class": "property_title prominent "})
        assert len(urls) == len(names)
        for i,d in enumerate(urls):
            name = names[i].get_text(strip=True)
            rev_num = int(d.get_text(strip=True).split()[0].replace(',', ''))
            url = d.get("href")
            list_hotel.append((name, url, rev_num))
    list_hotel.sort(key=lambda x: x[2], reverse=True)
    return list_hotel
hotel_links = hotel_urls(region_links)
hotel_links[:5]

[('International Hotel and Spa Calgary',
  '/Hotel_Review-g154913-d182256-Reviews-International_Hotel_and_Spa_Calgary-Calgary_Alberta.html#REVIEWS',
  3065),
 ('The Fairmont Palliser',
  '/Hotel_Review-g154913-d185694-Reviews-The_Fairmont_Palliser-Calgary_Alberta.html#REVIEWS',
  2701),
 ('Hyatt Regency Calgary',
  '/Hotel_Review-g154913-d184252-Reviews-Hyatt_Regency_Calgary-Calgary_Alberta.html#REVIEWS',
  1806),
 ('Best Western Premier Freeport Inn & Suites',
  '/Hotel_Review-g154913-d1974054-Reviews-Best_Western_Premier_Freeport_Inn_Suites-Calgary_Alberta.html#REVIEWS',
  1672),
 ('The Westin Calgary',
  '/Hotel_Review-g154913-d155238-Reviews-The_Westin_Calgary-Calgary_Alberta.html#REVIEWS',
  1585)]

In [4]:
hotel_link = tripadvisorca + hotel_links[0][1]
def review_urls(hotel_link):
    result = requests.get(hotel_link)
    content = result.content
    soup = BeautifulSoup(content, "html.parser")
    num_pages = int(soup.find_all("a", attrs={"class":"pageNum last taLnk "})[0].get_text(strip=True))
    list_urls = []
    for i in range(num_pages):
        bn = "-Reviews-or" + str(i * 5) + "-"
        url = hotel_link.replace("-Reviews-", bn)
        list_urls.append(url)
    return list_urls
review_links = review_urls(hotel_link)
review_links[:5]

['https://www.tripadvisor.ca/Hotel_Review-g154913-d182256-Reviews-or0-International_Hotel_and_Spa_Calgary-Calgary_Alberta.html#REVIEWS',
 'https://www.tripadvisor.ca/Hotel_Review-g154913-d182256-Reviews-or5-International_Hotel_and_Spa_Calgary-Calgary_Alberta.html#REVIEWS',
 'https://www.tripadvisor.ca/Hotel_Review-g154913-d182256-Reviews-or10-International_Hotel_and_Spa_Calgary-Calgary_Alberta.html#REVIEWS',
 'https://www.tripadvisor.ca/Hotel_Review-g154913-d182256-Reviews-or15-International_Hotel_and_Spa_Calgary-Calgary_Alberta.html#REVIEWS',
 'https://www.tripadvisor.ca/Hotel_Review-g154913-d182256-Reviews-or20-International_Hotel_and_Spa_Calgary-Calgary_Alberta.html#REVIEWS']

In [78]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
opts = Options()
opts.set_headless()
opts.set_preference("permissions.default.image", 2)
assert opts.headless  # Operating in headless mode

review_link = review_links[0]
def reviews_data(review_link):
    data = []
    now = time.time()
#     browser = webdriver.Firefox(options=opts)
    browser = webdriver.Firefox()
    browser.implicitly_wait(10)
    browser.get(review_link)
    review_zone = browser.find_element_by_id("taplc_location_reviews_list_resp_hr_resp_0")
    #find the first button
#     first_btn = review_zone.find_element_by_class_name("ulBlueLinks")
    first_btn = review_zone.find_element_by_css_selector("div.prw_rup.prw_reviews_text_summary_hsx > div > p > span")
    first_btn.click()
    time.sleep(1)
    revs = review_zone.find_elements_by_css_selector("div.ui_column.is-9 > div.prw_rup.prw_reviews_text_summary_hsx > div > p")
    dats = review_zone.find_elements_by_css_selector("div.ui_column.is-9 > span.ratingDate")
    rats = review_zone.find_elements_by_css_selector("div.ui_column.is-9 > span.ui_bubble_rating")
    assert len(dats) == len(rats) == len(revs) == 5
    for i, rev in enumerate(revs):
        data.append((dats[i].get_attribute("title"),rev.text, rats[i].get_attribute("class").split()[-1].split("_")[-1]))
    browser.quit()
    print("Streaming %d reviews took %.4f (seconds)" % (len(data),time.time()-now))
    return data
data = reviews_data(review_links[0])
data

Streaming 5 reviews took 9.3499 (seconds)


[('September 14, 2018',
  'Elevator was not working and blocked for luggage movement, No proper service for Guest. I had to wait for 20 minutes each day for elevator and delay to office each day. I came on business travel and each day because of the elevator issue had to reach office (banker hall) late. Poor elevator service.',
  '20'),
 ('September 13, 2018',
  'A great place to stay if traveling on on business to Calgary, as very quick and easy access to the airport and the main Financial District. Spacious suites, with outdoor space, and friendly reception.',
  '50'),
 ('September 9, 2018',
  'Yes this hotel is under construction.Very noisy, and we ended up with an old room.Very pricey for a room that had a very hard bed and the lights in the bathroom didnt work.This was the worst business trip we have been on.',
  '20'),
 ('September 9, 2018',
  'Everyone can understand a renovation will impose some challenges. But that’s not what made me swap hotels- International Hotel is a littl

In [87]:
opts = Options()
print(opts.headless)
opts.set_headless()
print(opts.headless)
opts.set_preference("--headless", False)
print(opts.headless)

False
True
True


In [46]:
review_link = "https://www.tripadvisor.ca/Hotel_Review-g154913-d182256-Reviews-or285-International_Hotel_and_Spa_Calgary-Calgary_Alberta.html"
review_link = 'https://www.tripadvisor.ca/Hotel_Review-g154913-d182256-Reviews-or0-International_Hotel_and_Spa_Calgary-Calgary_Alberta.html#REVIEWS'
data = []
browser = webdriver.Firefox()
browser.implicitly_wait(10)
browser.get(review_link)
review_zone = browser.find_element_by_id("taplc_location_reviews_list_resp_hr_resp_0")

In [47]:
now = time.time()
try:
    more_buttons = review_zone.find_elements_by_css_selector("div.prw_rup.prw_reviews_text_summary_hsx > div > p > span")
    if len(more_buttons) > 0:
        more_buttons[0].click()
        time.sleep(.1)
    print(time.time()-now)
except:
    print("no 'more' button")
    print(time.time()-now)

0.8380601406097412


In [58]:
#review_616230222 > div > div.ui_column.is-9 > span.ratingDate
#review_616230222 > div > div.ui_column.is-9 > span.ratingDate
#review_616230222 > div > div.ui_column.is-9 > span.ui_bubble_rating.bubble_50
dats = review_zone.find_elements_by_css_selector("div.ui_column.is-9 > span.ratingDate")
dat = dats[0]
print(len(dats))
dat.get_attribute("title")

5


'September 13, 2018'

In [88]:
from datetime import datetime
datetime.strptime(dat.get_attribute("title"), '%B %d, %Y')

AttributeError: module 'datetime' has no attribute 'strptime'

In [59]:
rats = review_zone.find_elements_by_css_selector("div.ui_column.is-9 > span.ui_bubble_rating")
rat = rats[0]
len(rats)


5

In [75]:
rat.get_attribute("class").split()[-1].split("_")[-1]

'50'

In [77]:
browser.quit()

In [ ]:
#find the first button
# first_btn = review_zone.("ulBlueLinks")
first_btn = review_zone.find_element_by_css_selector("div.prw_rup.prw_reviews_text_summary_hsx > div > p > span")
first_btn.click()
time.sleep(1)
revs = review_zone.find_elements_by_css_selector("div.ui_column.is-9 > div.prw_rup.prw_reviews_text_summary_hsx > div > p")
assert len(revs) == 5
for rev in revs:
    data.append(rev.text)

In [ ]:
//*[@id="review_615263430"]/div/div[2]/div[2]/div/p/span
//*[@id="review_616230222"]/div/div[2]/div[2]/div/p/span
#review_616230222 > div > div.ui_column.is-9 > div.prw_rup.prw_reviews_text_summary_hsx > div > p > span
//*[@id="taplc_location_reviews_list_resp_hr_resp_0"]

In [45]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

review_link = review_links[0] # page i + 1

data = []
# browser = webdriver.Firefox(options=opts)
browser = webdriver.Firefox()
wait = WebDriverWait(browser, 10)
browser.implicitly_wait(10)
browser.get(review_link)
review_zone = browser.find_element_by_id("taplc_location_reviews_list_resp_hr_resp_0")
#find the first button
wait.until_not(EC.visibility_of_element_located((By.CLASS_NAME, "tabs_pers_titles")))
more_buttons = review_zone.find_elements_by_css_selector("div.prw_rup.prw_reviews_text_summary_hsx > div > p > span")
if len(more_buttons) > 0:
    more_buttons[0].click()
    time.sleep(.1)
# review_zone.find_element_by_css_selector("div.prw_rup.prw_reviews_text_summary_hsx > div > p > span").click()
wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'div.ui_column.is-9 > div.prw_rup.prw_reviews_text_summary_hsx > div > p')))
revs = review_zone.find_elements_by_css_selector("div.ui_column.is-9 > div.prw_rup.prw_reviews_text_summary_hsx > div > p")
assert len(revs) == 5
for rev in revs:
    data.append(rev.text)
    
num_pages = int(review_zone.find_element_by_css_selector("a.pageNum.last.taLnk").text)
# num_pages = 3
i = 1
while i<num_pages:
    i += 1
    try:
        # located next button and clicked on it
        review_zone.find_element_by_css_selector("a.nav.next.taLnk.ui_button.primary").click()
        # wait until the new content is loaded
        try:
            wait.until_not(EC.visibility_of_element_located((By.ID, "taplc_hotels_loading_box_hr_resp_0")))
        except:
            print("Page %d is reloaded"%i-1)
            time.sleep(1)
            continue
            
        try:
            wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'div.ui_column.is-9 > div.prw_rup.prw_reviews_text_summary_hsx > div > p > span')))
            more_buttons = review_zone.find_elements_by_css_selector("div.prw_rup.prw_reviews_text_summary_hsx > div > p > span")
            if len(more_buttons) > 0:
                more_buttons[0].click()
                time.sleep(.1)
#             review_zone.find_element_by_css_selector("div.prw_rup.prw_reviews_text_summary_hsx > div > p > span").click()
        except:
            # In case of not having a <more> field
            print("Page %d doesn't have the 'more' section"%i)
            time.sleep(1)
            continue
            
        try:
            wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'div.ui_column.is-9 > div.prw_rup.prw_reviews_text_summary_hsx > div > p')))
            revs = review_zone.find_elements_by_css_selector("div.ui_column.is-9 > div.prw_rup.prw_reviews_text_summary_hsx > div > p")
            dats = review_zone.find_elements_by_css_selector("div.ui_column.is-9 > span.ratingDate")
            rats = review_zone.find_elements_by_css_selector("div.ui_column.is-9 > span.ui_bubble_rating")
            assert len(dats) == len(rats) == len(revs) == 5
            for i, rev in enumerate(revs):
                data.append((dats[i].get_attribute("title"),rev.text, rats[i].get_attribute("class").split()[-1].split("_")[-1]))

        except:
            # In case of not having a <more> field
            print("Page %d doesn't have the 'p' section"%i)
            time.sleep(1)
            revs = review_zone.find_elements_by_css_selector("div.ui_column.is-9 > div.prw_rup.prw_reviews_text_summary_hsx > div > p")
            dats = review_zone.find_elements_by_css_selector("div.ui_column.is-9 > span.ratingDate")
            rats = review_zone.find_elements_by_css_selector("div.ui_column.is-9 > span.ui_bubble_rating")
            assert len(dats) == len(rats) == len(revs) == 5
            for i, rev in enumerate(revs):
                data.append((dats[i].get_attribute("title"),rev.text, rats[i].get_attribute("class").split()[-1].split("_")[-1]))

            continue
            
    except:
        print("Can't scrape the reviews at page %d"%i)
        time.sleep(1)
        continue
        
browser.quit()

Page 58 doesn't have a more section
Page 98 doesn't have a 'p' section
Page 167 doesn't have a more section
Page 293 doesn't have a more section
Page 325 doesn't have a more section
Page 382 doesn't have a more section
Page 406 doesn't have a more section
Page 434 doesn't have a more section
Can't read the reviews at page 435
Can't read the reviews at page 436
Can't read the reviews at page 437
Can't read the reviews at page 438
Can't read the reviews at page 439
Can't read the reviews at page 440
Can't read the reviews at page 441
Can't read the reviews at page 442
Can't read the reviews at page 443
Can't read the reviews at page 444
Can't read the reviews at page 445
Can't read the reviews at page 446
Can't read the reviews at page 447
Can't read the reviews at page 448
Can't read the reviews at page 449
Can't read the reviews at page 450
Can't read the reviews at page 451
Can't read the reviews at page 452
Can't read the reviews at page 453
Can't read the reviews at page 454
Can't r

In [ ]:
#review_604831330 > div > div.ui_column.is-9 > div.prw_rup.prw_reviews_text_summary_hsx > div > p
#review_492466432 > div > div.ui_column.is-9 > div.prw_rup.prw_reviews_text_summary_hsx > div > p

In [116]:
browser.quit()

In [70]:
nxt = review_zone.find_element_by_css_selector("a.nav.next.taLnk.ui_button.primary")
nxt.click()

In [71]:
first_btn = review_zone.find_element_by_class_name("ulBlueLinks")
first_btn.click()
browser.implicitly_wait(2)

In [72]:
revs = review_zone.find_elements_by_class_name("ui_column.is-9")
revs = review_zone.find_elements_by_css_selector("div.ui_column.is-9 > div.prw_rup.prw_reviews_text_summary_hsx > div > p")
assert len(revs) == 5
for rev in revs:
    data.append(rev.text)

In [21]:
#taplc_location_reviews_list_resp_hr_resp_0 > div > div:nth-child(10) > div > div > div > a.pageNum.last.taLnk
#taplc_location_reviews_list_resp_hr_resp_0 > div > div:nth-child(10) > div > div > a.nav.next.taLnk.ui_button.primary
#taplc_location_reviews_list_resp_hr_resp_0 > div > div:nth-child(10) > div > div > a.nav.next.taLnk.ui_button.primary
#taplc_location_reviews_list_resp_hr_resp_0 > div > div:nth-child(10) > div > div > a.nav.next.taLnk.ui_button.primary

In [113]:
review_zone.find_elements_by_css_selector("div.ui_column.is-9 > div.prw_rup.prw_reviews_text_summary_hsx > div > p")

[]